In [1]:
import pickle
import re
import emoji
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import numpy as np

# Load tokenizer, label encoder, vocabulary size, and stop words
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('label_encoder.pickle', 'rb') as handle:
    le = pickle.load(handle)

with open('vocabSize.pickle', 'rb') as handle:
    vocab_size = pickle.load(handle)

with open('stopwords.pickle', 'rb') as handle:
    stop_words = pickle.load(handle)

# Load pre-trained model
model = load_model('my_model91.h5')

def replace_emojis_with_text(text):
    return emoji.demojize(text)

def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(y) for y in text]
    return " ".join(text)

def remove_stop_words(text):
    Text = [i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

def removing_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

def lower_case(text):
    text = text.split()
    text = [y.lower() for y in text]
    return " ".join(text)

def removing_punctuations(text):
    ## Remove punctuations
    text = re.sub(r'[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@^_`{|}~"""), ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = " ".join(text.split())
    return text.strip()

def removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan


def normalized_sentence(sentence):
    sentence = replace_emojis_with_text(sentence)  # Adding emoji replacement
    sentence = lower_case(sentence)
    sentence = remove_stop_words(sentence)
    sentence = removing_numbers(sentence)
    sentence = removing_punctuations(sentence)
    sentence = removing_urls(sentence)
    sentence = lemmatization(sentence)
    return sentence

def predict_sentiment(sentence):
    # Normalize the sentence
    sentence = normalized_sentence(sentence)

    # Tokenize the normalized sentence
    sentence = tokenizer.texts_to_sequences([sentence])

    # Pad the tokenized sentence
    sentence = pad_sequences(sentence, maxlen=229, truncating='pre')

    # Predict using the model
    result = le.inverse_transform(np.argmax(model.predict(sentence), axis=-1))[0]
    proba =  np.max(model.predict(sentence))
    return result, proba

# Example usage
sentence = "I'm feeling sad today"
result, proba = predict_sentiment(sentence)
print(f"Predicted sentiment: {result} with probability: {proba}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
Predicted sentiment: 0 with probability: 0.9998694658279419


In [2]:
def predict_sentiment(sentence):
    # Normalize the sentence
    sentence = normalized_sentence(sentence)

    # Tokenize the normalized sentence
    sentence = tokenizer.texts_to_sequences([sentence])

    # Pad the tokenized sentence
    sentence = pad_sequences(sentence, maxlen=229, truncating='pre')

    # Predict using the model
    predictions = model.predict(sentence)
    predicted_emotion = le.inverse_transform(np.argmax(predictions, axis=-1))[0]
    max_proba = np.max(predictions)
    
    # Get probabilities for all emotions
    probabilities = {emotion: proba for emotion, proba in zip(le.classes_, predictions[0])}
    
    return predicted_emotion, max_proba, probabilities

# Example usage:
sentence = "I feel so happy today!"
predicted_emotion, max_proba, probabilities = predict_sentiment(sentence)
print("Predicted Emotion:", predicted_emotion)
print("Max Probability:", max_proba)
print("Probabilities for all emotions:", probabilities)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
Predicted Emotion: 1
Max Probability: 0.99983704
Probabilities for all emotions: {0: 4.1988198e-05, 1: 0.99983704, 2: 8.0529164e-05, 3: 4.451199e-06, 4: 7.847806e-06, 5: 2.813392e-05}


In [3]:
emoDict = {
    0:'sadness',
    1:'joy',
    2:'fear',
    3:'anger',
    4:'love',
    5:'surprise'
}

In [4]:
sentence = "I'm feeling angry today"
# result, max_probaproba = predict_sentiment(sentence)
# print(f"Predicted sentiment: {emoDict[result]} || with probability: {proba}")
predicted_emotion, max_proba, probabilities = predict_sentiment(sentence)
print("Predicted Emotion:", predicted_emotion)
print("Max Probability:", max_proba)
print("Probabilities for all emotions:", probabilities)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
Predicted Emotion: 3
Max Probability: 0.9997099
Probabilities for all emotions: {0: 8.255614e-05, 1: 2.4001707e-05, 2: 3.9154802e-05, 3: 0.9997099, 4: 0.00013732266, 5: 7.022392e-06}


In [5]:
sentence = "I was in ❤️ with her"
# result, proba = predict_sentiment(sentence)
# print(f"Predicted sentiment: {emoDict[result]} || with probability: {proba}")
predicted_emotion, max_proba, probabilities = predict_sentiment(sentence)
print("Predicted Emotion:", predicted_emotion)
print("Max Probability:", max_proba)
print("Probabilities for all emotions:", probabilities)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
Predicted Emotion: 4
Max Probability: 0.7244732
Probabilities for all emotions: {0: 0.067867294, 1: 0.035041153, 2: 0.0032625268, 3: 0.16212554, 4: 0.7244732, 5: 0.0072303647}


In [6]:
from ntscraper import Nitter
scraper = Nitter()

Testing instances: 100%|███████████████████████████████████████████████████████████████| 77/77 [04:04<00:00,  3.18s/it]


In [7]:
tweets = scraper.get_tweets('elonmusk', mode='user',number=100)

INFO:root:No instance specified, using random instance https://nitter.privacydev.net
INFO:root:Current stats for elonmusk: 20 tweets, 0 threads...
INFO:root:Current stats for elonmusk: 40 tweets, 0 threads...
INFO:root:Current stats for elonmusk: 60 tweets, 0 threads...
INFO:root:Current stats for elonmusk: 80 tweets, 0 threads...
INFO:root:Current stats for elonmusk: 100 tweets, 0 threads...


In [8]:
#Alternative Solution using snscrape
posts = []
count=1
for tweet in tweets['tweets']:
  posts.append(tweet['text'])


In [13]:
count = 1
for text in posts:
    # result, proba = predict_sentiment(text)
    print(count,text)
    
    # print(f"{count}. Predicted sentiment: {emoDict[result]} || with probability: {proba}")
    predicted_emotion, max_proba, probabilities = predict_sentiment(text)
    print(f"Predicted Emotion: {predicted_emotion} {emoDict[predicted_emotion]}")
    print("Max Probability:", max_proba)
    print("Probabilities for all emotions:", probabilities)
    count+=1

1 Just a reminder that Earth is almost empty of humans, contrary to what many people think https://waitbutwhy.com/2015/03/7-3-billion-people-one-building.html
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
Predicted Emotion: 3 anger
Max Probability: 0.49685937
Probabilities for all emotions: {0: 0.15072754, 1: 0.13075271, 2: 0.008899616, 3: 0.49685937, 4: 0.20289677, 5: 0.009863965}
2 From 100,000 miles away 📸   This Apollo 10 image of Earth was taken 55 years ago in 1969.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
Predicted Emotion: 1 joy
Max Probability: 0.30148205
Probabilities for all emotions: {0: 0.21908157, 1: 0.30148205, 2: 0.011051687, 3: 0.19023706, 4: 0.26224127, 5: 0.015906338}
3 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Predicted Emotion: 3 anger
Max Probability: 0.4130388
Probabilities for all emotions: {0: 0.112072855, 1: 0.11115428, 2: 0.0149275875, 3: 0.4130388, 4: 0.33302134, 5: 0.0157852}
4 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Predicted Emotion: 3 anger
Max Probability: 0.41303

In [10]:
count = 1
for text in posts:
    print(count,text)
    count+=1
    print()



1 Just a reminder that Earth is almost empty of humans, contrary to what many people think https://waitbutwhy.com/2015/03/7-3-billion-people-one-building.html

2 From 100,000 miles away 📸   This Apollo 10 image of Earth was taken 55 years ago in 1969.

3 

4 

5 Anything that potentially challenges the Standard Model is a very interesting experiment.   Wild that Germanium-76 takes 100 trillion times the age of the universe to decay, yet it is observable in a year in you have ~1 ton of it!  https://newscenter.lbl.gov/2012/05/16/majorana-demonstrator/

6 “Tesla is far ahead in self-driving cars” – Jensen Huang

7 

8 The goal of citizenship for all illegals is very clearly stated by the leader of the Senate.  For some reason, a lot of people still think this is some crazy conspiracy theory!

9 WhatsApp exports your user data every night.   Some people still think it is secure.

10 Starship is designed to fundamentally alter humanity’s access to space, ultimately enabling us to make life 